In [21]:
!pip install speechbrain
!pip install torchaudio


  Using cached sentencepiece-0.2.0-cp312-cp312-win_amd64.whl.metadata (8.3 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/864.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/864.1 kB ? eta -:--:--
   ------------------------ --------------- 524.3/864.1 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 864.1/864.1 kB 1.7 MB/s eta 0:00:00
Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl (156 kB)
Using cached sentencepiece-0.2.0-cp312-cp312-win_amd64.whl (991 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.5 MB 1.9 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.5 MB 1.9 MB/s eta 0:00:01
   ------------------------- -------------- 1.6/2.5 MB 2.0 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 

In [22]:
import os
import librosa
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from tqdm import tqdm
from speechbrain.pretrained import SpeakerRecognition
import torchaudio


c:\Users\Rishi S Etagi\Desktop\medivoice\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Python312\Lib\inspect.py:1004: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
C:\Users\Rishi S Etagi\AppData\Local\Temp\ipykernel_4276\2983932696.py:7: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import SpeakerRecognition


In [23]:
AUDIO_DIR = r"C:\Users\Rishi S Etagi\Desktop\medivoice\LIBRI"

OUTPUT_CSV = "speech_features.csv"

In [24]:
def predict_gender(filepath):
    try:
        y, sr = librosa.load(filepath, sr=16000)
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
        pitch = pitches[magnitudes > np.median(magnitudes)]
        pitch = pitch[pitch > 0]
        pitch_mean = np.mean(pitch) if len(pitch) > 0 else 0
        return "male" if pitch_mean < 165 else "female"
    except Exception as e:
        print(f"Gender detection error in {filepath}: {e}")
        return "unknown"
    
def extract_features(filepath):
    y, sr = librosa.load(filepath, sr=16000)

    # MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfccs, axis=1)

    # Pitch-related features
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch = pitches[magnitudes > np.median(magnitudes)]
    pitch = pitch[pitch > 0]  # Filter out zeros

    pitch_mean = np.mean(pitch) if len(pitch) > 0 else 0
    pitch_min = np.min(pitch) if len(pitch) > 0 else 0
    pitch_max = np.max(pitch) if len(pitch) > 0 else 0
    pitch_depth = pitch_max - pitch_min

    # Energy
    energy = np.sum(librosa.feature.rms(y=y))

    # Speaking Rate
    envelope = np.abs(y)
    peaks, _ = find_peaks(envelope, height=0.02, distance=1000)
    duration = len(y) / sr
    speaking_rate = len(peaks) / duration if duration > 0 else 0

    gender = predict_gender(filepath)

    return {
        "filename": os.path.basename(filepath),
        "pitch_mean": pitch_mean,
        "pitch_min": pitch_min,
        "pitch_max": pitch_max,
        "pitch_depth": pitch_depth,
        "energy": energy,
        "speaking_rate": speaking_rate,
        "gender": gender,
        **{f"mfcc_{i+1}": mfcc_mean[i] for i in range(len(mfcc_mean))}
    }

# Extraction loop
features = []
for root, dirs, files in os.walk(AUDIO_DIR):
    for file in tqdm(files):
        if file.endswith(".flac"):
            path = os.path.join(root, file)
            try:
                f = extract_features(path)
                features.append(f)
            except Exception as e:
                print(f"Error processing {file}: {e}")


100%|██████████| 2703/2703 [01:14<00:00, 36.23it/s]


In [25]:
df = pd.DataFrame(features)
df.to_csv(OUTPUT_CSV, index=False)
